<a href="https://colab.research.google.com/github/SreejaKondaparthi/Major-Project/blob/main/Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi || echo "No NVIDIA GPU detected. Set Runtime → Change runtime type → GPU."

Wed Nov  5 18:56:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   42C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%bash
pip -q install --no-cache-dir -U \
  "ultralytics>=8.3.0,<9" \
  "roboflow"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 238.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 274.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 289.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 334.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 391.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 355.0 MB/s eta 0:00:00


In [3]:
import PIL, datasets, huggingface_hub, pyarrow, ultralytics, tqdm
print("Pillow:", PIL.__version__)
import huggingface_hub as hfh; print("HF Hub:", hfh.__version__)
import datasets as ds; print("datasets:", ds.__version__)
import pyarrow as pa; print("pyarrow:", pa.__version__)
import ultralytics; print("ultralytics:", ultralytics.__version__)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Pillow: 11.3.0
HF Hub: 0.36.0
datasets: 4.0.0
pyarrow: 18.1.0
ultralytics: 8.3.225


In [4]:
REPO_URL = "https://github.com/SreejaKondaparthi/Major-Project.git"
!rm -rf Major-Project
!git clone {REPO_URL}
%cd Major-Project
!ls -la

Cloning into 'Major-Project'...
remote: Enumerating objects: 7963, done.
remote: Total 7963 (delta 0), reused 0 (delta 0), pack-reused 7963 (from 1)
Receiving objects: 100% (7963/7963), 789.71 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Updating files: 100% (8264/8264), done.
Filtering content: 100% (1464/1464), 745.21 MiB | 8.75 MiB/s, done.
/content/Major-Project
total 36
drwxr-xr-x 7 root root 4096 Nov  5 18:57 .
drwxr-xr-x 1 root root 4096 Nov  5 18:56 ..
drwxr-xr-x 5 root root 4096 Nov  5 18:57 data_raw
-rw-r--r-- 1 root root  144 Nov  5 18:57 dataset.yaml
drwxr-xr-x 4 root root 4096 Nov  5 18:57 data_yolo
drwxr-xr-x 9 root root 4096 Nov  5 18:59 .git
-rw-r--r-- 1 root root  320 Nov  5 18:57 .gitignore
drwxr-xr-x 3 root root 4096 Nov  5 18:57 runs
drwxr-xr-x 2 root root 4096 Nov  5 18:57 tools


In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="xplwaup7cpwBLLcMVi0i")
project = rf.workspace("rico-tqjvo").project("rico-c9kwc")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rico-1 in yolov8:: 100%|██████████| 2550/2550 [00:00<00:00, 9336.91it/s]


In [7]:
from ultralytics import YOLO

# --- Define Paths ---
# Use the standard pre-trained YOLOv8m model as a starting point (Ultralytics downloads this)
BEST_WEIGHTS = 'yolov8m.pt'
# Data path from the Roboflow download
DATA_YAML = '/content/Major-Project/Rico-1/data.yaml'

# 1. Load the base YOLOv8 model
model = YOLO(BEST_WEIGHTS)

# 2. Start training with optimized hyperparameters
# We increase epochs to 150 since we are not starting from your specialized weights
model.train(
    data=DATA_YAML,
    imgsz=1280,       # High resolution for small components
    epochs=150,       # Significantly more epochs for deep learning
    batch=4,          # Reduced batch size for 1280px
    device=0,         # Use the GPU
    cache='disk',
    # Hyperparameters for addressing rare classes and generalization:
    copy_paste=0.2,   # Adds instances of rare components
    scale=0.5,
    translate=0.05,
    # Naming the run clearly
    name='rico_roboflow_final_y8m_150e_from_base'
)

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Major-Project/Rico-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rico_roboflow_final_y8m_150e_from_base2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patien

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e845c928ef0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0